# 使用 LCEL 构建一个简单的 LLM 应用程序
在本快速入门中，我们将向您展示如何使用 LangChain 构建一个简单的 LLM 应用程序。此应用程序会将文本从英语翻译成另一种语言。这是一个相对简单的 LLM 应用程序 - 它只是一个 LLM 调用加上一些提示。不过，这仍然是开始使用 LangChain 的好方法 - 只需一些提示和一个 LLM 调用就可以构建许多功能！

In [1]:
from dotenv import load_dotenv,find_dotenv
import os 

# 获取你的智谱 API Key
在当前文件下创建一个.env文件，将api-key复制进去，如ZHIPUAI_API_KEY = "api-key"

In [2]:
_ = load_dotenv(find_dotenv())

# Using Language Models

首先，让我们学习如何单独使用语言模型。LangChain 支持多种不同的语言模型，您可以互换使用 - 在下面选择您想要使用的模型！

In [3]:
from langchain_community.chat_models import ChatZhipuAI

model = ChatZhipuAI(
    model="glm-4",
    temperature=0.5
)

让我们首先直接使用该模型。ChatModel 是 LangChain“Runnable”的实例，这意味着它们公开了一个用于与其交互的标准接口。为了简单地调用该模型，我们可以将消息列表传递给 .invoke 方法。

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Chinese"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='你好！', response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 16, 'total_tokens': 20}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-a5e1419d-9bc6-4e0b-961a-6554e0334533-0')

## OutputParsers-输出解析器

请注意，模型的响应是 AIMessage。它包含字符串响应以及有关响应的其他元数据。通常我们可能只想处理字符串响应。我们可以使用简单的输出解析器来解析此响应。  

我们首先导入简单的输出解析器。

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [6]:
result = model.invoke(messages)

In [7]:
parser.invoke(result)

'你好！'

更常见的是，我们可以将模型与此输出解析器“链接”在一起。这意味着此输出解析器将在此链中每次被调用。此链采用语言模型的输入类型（字符串或消息列表）并返回输出解析器的输出类型（字符串）。  

我们可以使用 | 运算符轻松创建链。| 运算符在 LangChain 中用于将两个元素组合在一起。

In [8]:
chain = model | parser

In [9]:
chain.invoke(messages)

'你好！'

# Prompt Templates - 提示模板

现在，我们将消息列表直接传递到语言模型中。这个消息列表从何而来？通常，它由用户输入和应用程序逻辑组合而成。此应用程序逻辑通常获取原始用户输入并将其转换为可传递给语言模型的消息列表。常见的转换包括添加系统消息或使用用户输入格式化模板。  

PromptTemplates 是 LangChain 中的一个概念，旨在协助进行这种转换。它们接收原始用户输入并返回可传递给语言模型的数据（提示）。  

让我们在这里创建一个 PromptTemplate。它将接收两个用户变量：  

 - language：要将文本翻译成的语言
 - text：要翻译的文本

In [10]:
from langchain_core.prompts import ChatPromptTemplate

首先，让我们创建一个字符串，将其格式化为系统消息：

In [11]:
system_template = "Translate the following into {language}:"

接下来，我们可以创建 PromptTemplate。这将是 system_template 的组合，以及用于放置文本的更简单的模板

In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

此提示模板的输入是dict对象。我们可以单独试用此提示模板，看看它自己能做什么

In [14]:
result = prompt_template.invoke({"language": "Chinese", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Chinese:'), HumanMessage(content='hi')])

我们可以看到它返回了一个由两条消息组成的 ChatPromptValue。如果我们想直接访问这些消息，我们可以这样做：

In [15]:
result.to_messages()

[SystemMessage(content='Translate the following into Chinese:'),
 HumanMessage(content='hi')]

# Chaining together components with LCEL - 使用 LCEL 将组件链接在一起

我们现在可以使用管道（|）运算符将其与上面的模型和输出解析器结合起来：

In [16]:
chain = prompt_template | model | parser

In [19]:
chain.invoke({"language": "chinese", "text": "hi"})

'您好！但是您提供的内容只有“hi”，这是一个非常短的问候语，它的中文翻译是“嗨”。如果您有更长的文本需要翻译，请提供，我会很乐意帮助您。'